This code is written in R, we need to seeting R kernel in Jupyter notebook.
Time:2023-12-20

# 1. Seeting R in Jupyter notebook
https://docs.anaconda.com/free/navigator/tutorials/r-lang/

Step 1: Open the Anaconda Prompt
To start, open the Anaconda Prompt.

You’ll then see the following screen with your user name:
(base) C:\Users\Ron>

Step 2: Add R to Jupyter
Next, type/copy the following command in order to add R to Jupyter:

'conda install -c r r-irkernel'

The command would look like this in the Anaconda Prompt (press ENTER to proceed with the installation):

'(base) C:\Users\Ron>conda install -c r r-irkernel'

Next, type ‘y‘ and then press ENTER to proceed:

'Proceed ([y]/n)? y'

After a short period of time, your installation would be completed.

Step 3: Launch Jupyter Notebook
To launch Jupyter Notebook, first open the Anaconda Navigator.

Then, click on the button to launch Jupyter.

Step 4: Create a new Notebook
To create a new Notebook for R:

First, click on 'New' on the top right-hand-side of your screen
Finally, select 'R' from the drop-down list

## If the computer has installed R software, seeting Jupyter notebook in R
Install package in R directly.

install.packages('IRkernel')
IRkernel::installspec()

# 2. Run the following code

Step 1: Install the R package.

Step 2: Setting up the folder.

Step 3: Read NDVI data from computer.

Step 4: Trend analysis of Sen+MK long time series.

Step 5: Display Outputs

## Step 1: Install the R packages.

In [2]:
#Install the R package
#install.packages("terra")
#install.packages("trend")

library(terra)
library(trend)

terra 1.7.65



## Step 2: Setting up the folder.

In [3]:
#Check the current folder
getwd()
#Setting the Run Folder
setwd("D:/Code/R/NDVI_RAIN/")

[1] "C:/Users/Qing/Sen+MK _BBNP"

## Step 3: Read NDVI data from computer.
Enter a folder of single-band TIFF data, in this case the NDVI annual maximum/mean for the calendar year

Data detail:

1) Modis_NDVI
Source: LAADS DAAC, MODIS MOD13Q1 NDVI gridded level-3 product, 250m resolution, 16 day composites.

In the file './Mon/Modis_NDVI', it contains monthly data from 2013 to 2022, i.e. 20 years and 12 months.

2) Sentinel2_NDVI
Source: Copernicus Sentinel-2 Collection 1 MSI Level-2A (L2A) product, 10m resolution. 

In the file './Mon/Sentinel2_NDVI', it contains quarterly data from 2017 to 2022, i.e. 6 years and 2 seasons (summer and autumn).

In [4]:
flnames <- list.files(path = './Mon/Modis_NDVI/NDVI_1/', pattern = '.tif$')
   #Modis NDVI from 2003-2022, for 20 years.
   #'NDVI_1'=January, we need to manually change the month or season
fl <- paste0("./Mon/Modis_NDVI/NDVI_1/", flnames)
firs <- rast(fl)

ERROR: Error: [rast] extents do not match


In [5]:
##The operator console displays all loaded data
plot(firs)
plot(firs[1])#show first image
summary(firs)
ts = global(firs, "mean", na.rm=T, cores=8)#core=Computer Kernel 8
a1 = sens.slope(ts$mean)

ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'plot': object 'firs' not found


## Step 4: Trend analysis of Sen+MK long time series.
#Sen+MK analysis
Change the time length of the data, 'start = begining year'  and 'end = ending year'.

1) Theil-Sen Median

> The Theil-Sen Median method, also known as Sen slope estimation, is a robust non-parametric statistical approach for trend analysis. The method is computationally efficient, insensitive to measurement error and outlier data, and is suitable for trend analysis of long time series data

2）Mann-Kendall Test

> The Mann-Kendall Test is used to determine whether a time series has a monotonic upward or downward trend. It does not require that the data be normally distributed or linear. It does require that there is no autocorrelation .

In [10]:
fun_sen <- function(x){
  if(length(na.omit(x)) <3) return(c(NA, NA, NA)) #Delete image elements with data discontinuities containing NA 
  MK_estimate <- trend::sens.slope(ts(na.omit(x), start = 2003, end = 2023, frequency = 1), conf.level = 0.95) #Sen slope
  slope <- MK_estimate$estimate
  MK_test <- MK_estimate$p.value
  Zs <- MK_estimate$statistic
  return(c(Zs, slope, MK_test))
}

firs_sen = app(firs, fun_sen, cores=8 )
names(firs_sen) = c("Z", "slope", "p-value")

## Step 5: Display Outputs
Plot the analysis results, include the "Z", "slope", "p-value".

"./Sen+MK/Month/" Setting up the write file.

Write results in raster, in "_January.tif" we need to manually change the file month or season.

In [1]:
plot(firs_sen)

writeRaster(firs_sen, filename = paste0("./Sen+MK/Month/", firs_sen@cpp[["names"]], "_Modis03-23_January.tif"))

ERROR: Error in eval(expr, envir, enclos): object 'firs_sen' not found
